In [1]:
import geopandas as gpd
import fiona
kml_file = "../files/CTO_113-11.kml"
kml_clean_file = "../files/Legend.kml"
import pandas as pd

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'


In [3]:
"""If you wanted to have all the folders of the kml file you can unccoment this block
and comment the one below"""

# df = pd.DataFrame()
# for layer in fiona.listlayers(kml_file):
#     s = gpd.read_file(kml_file, driver='KML', layer=layer)
#     df = pd.concat([df, s], ignore_index=True)

'If you wanted to have all the folders of the kml file you can unccoment this block\nand comment the one below'

In [3]:
# However, at this point we only want the folder with the utility poles.
df = gpd.read_file(kml_clean_file, driver='KML')

""" Since the type of the column "geometry" is Shapely.Point
    we have to change it manually."""
from shapely import wkt
df['str_geometry'] = df.geometry.apply(lambda x: wkt.dumps(x))
df['str_geometry'] = df.str_geometry.apply(lambda x: x[9:].split()[:2])
df['str_geometry'] = df.str_geometry.apply(lambda x: [x[1],x[0]])
df['latitude'] = df.str_geometry.apply(lambda x: x[0])
df['longitude'] = df.str_geometry.apply(lambda x: x[1])


In [6]:
""" It'd be a great idea to get the distances between every single utility pole
    before starting the actual algorithm"""
import geopandas as gpd
from geopy.distance import geodesic
from scipy.spatial.distance import pdist, squareform
coords = df[['latitude', 'longitude']].values
dist_matrix = list(squareform(pdist(coords, lambda u, v: geodesic(u, v).meters)))

array([ 134.93766365,    0.        ,  377.98850348,  650.75689376,
        742.85543941, 1087.78417886, 1225.19049465, 1362.00587653,
       1465.07426322, 1763.6481921 ,  917.26278127,  196.93035129,
        694.85692217,  585.60888157, 1541.8722096 , 1639.61588145,
       1590.42266435, 1462.80392791, 1193.36869494, 1085.3147106 ,
        960.77249075, 3291.82766179, 3006.3103674 , 2899.36230974,
       2792.83944996, 2959.77074126, 2823.42731986, 2692.97107315,
       2662.50172985, 3175.51942897, 3277.7120016 , 3386.48772585,
       3747.4596386 , 3769.13602885, 2365.85167173, 1864.45421104,
       1591.57697208, 1361.41611657, 1225.89960253, 1048.06350915,
        968.82967011,  925.91381772,  321.54155262, 4296.67712831,
       4496.67957012, 1360.92090874, 1600.58069794, 1130.36186292,
       2890.80416037, 2900.04923153, 2898.78957227, 2853.24914774,
       3103.62383609, 3121.16099559, 3024.92994733, 3205.2799285 ,
       3314.9713103 , 3342.37914265, 3296.11324163, 3372.20228

In [86]:
""" Now that we have a matrix, which each element Aij, represents the distance
    from pole i to pole j, we want to associate those distances with each pole
    """
poles_names = list(df.Name)
graph = {}
for i in range(len(poles_names)):
    graph[poles_names[i]] = []
    for j in range(len(poles_names)):
        if j == i: continue # We don't want to include distance from i to i, which is obviously 0.
        graph[poles_names[i]] += [(poles_names[j],dist_matrix[i][j])]

In [96]:
def get_edges(g):
    edges = []
    for count,ele in enumerate(g):
        for dest,weight in g[ele]:
            edges += [(ele,dest,weight)]
    return(sorted(edges, key = lambda x: x[2]))  
def kruskal(graph):
    edges = get_edges(graph)
    
    return []
a = get_edges(graph)


[('POSTE_113-11124', 'POSTE_113-11159', 2.8651355714296676),
 ('POSTE_113-11159', 'POSTE_113-11124', 2.8651355714296676),
 ('POSTE_113-11169', 'POSTE_113-11170', 2.870196147077509),
 ('POSTE_113-11170', 'POSTE_113-11169', 2.870196147077509),
 ('POSTE_113-11167', 'POSTE_113-11168', 2.981948056018276),
 ('POSTE_113-11168', 'POSTE_113-11167', 2.981948056018276),
 ('POSTE_113-11111', 'POSTE_113-11112', 4.1330794708308645),
 ('POSTE_113-11112', 'POSTE_113-11111', 4.1330794708308645),
 ('POSTE_113-1175', 'POSTE_113-11134', 6.508051498124716),
 ('POSTE_113-11134', 'POSTE_113-1175', 6.508051498124716),
 ('POSTE_113-11127', 'POSTE_113-11129', 8.038883734613615),
 ('POSTE_113-11129', 'POSTE_113-11127', 8.038883734613615),
 ('POSTE_113-1130', 'POSTE_113-11161', 8.712003589792864),
 ('POSTE_113-11161', 'POSTE_113-1130', 8.712003589792864),
 ('POSTE_113-11104', 'POSTE_113-11106', 9.963959196052349),
 ('POSTE_113-11106', 'POSTE_113-11104', 9.963959196052349),
 ('POSTE_113-1176', 'POSTE_113-11133', 1